<a href="https://colab.research.google.com/github/Zhmuda/Football_Data_Analytics/blob/main/ManCity_Passes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd

db_path = '/content/fbref_data.db'
conn = sqlite3.connect(db_path)

query = """
SELECT Player, Squad, Assists, Expected_Assists
FROM players_matches_stats
WHERE Squad = 'Manchester City'
"""

df = pd.read_sql(query, conn)
conn.close()

display(df.head())

,Player,Squad,Assists,Expected_Assists
0,Manuel Akanji,Manchester City,0,1.1
1,Nathan Aké,Manchester City,0,0.2
2,Kevin De Bruyne,Manchester City,6,4.6
3,Rúben Dias,Manchester City,0,0.8
4,Jeremy Doku,Manchester City,4,2.3


##Лучшие распасовщики: xA vs. Реальные ассисты
### Манчестер Сити

In [2]:
import plotly.express as px
import pandas as pd

# Приводим данные к числовому типу
df["Assists"] = pd.to_numeric(df["Assists"], errors="coerce")
df["Expected_Assists"] = pd.to_numeric(df["Expected_Assists"], errors="coerce")

# Определим одинаковые диапазоны для осей
x_range = [-0.1, df["Expected_Assists"].max() * 1.1]  # Немного расширим максимальное значение для оси X
y_range = [-0.1, df["Assists"].max() * 1.1]  # Аналогично для оси Y

# Создаем интерактивный график с именами игроков при наведении
fig = px.scatter(df, x="Expected_Assists", y="Assists",
                 hover_data={"Player": True, "Expected_Assists": True, "Assists": True},
                 labels={"Expected_Assists": "Ожидаемые ассисты (xA)", "Assists": "Реальные ассисты"})

# Добавляем линию xA = Assists
fig.add_shape(type="line", x0=0, y0=0, x1=df["Expected_Assists"].max(), y1=df["Expected_Assists"].max(),
              line=dict(color="red", dash="dash"))

# Обновляем layout
fig.update_layout(
    title="Сравнение xA и реальных ассистов для игроков Манчестер Сити",
    showlegend=False,
    xaxis_range=x_range,  # Устанавливаем диапазон оси X
    yaxis_range=y_range   # Устанавливаем диапазон оси Y
)

# Показываем график
fig.show()


Как анализировать этот график:
Линия xA = Assists: Красная пунктирная линия показывает, где ожидаемые ассисты (xA) равны реальным ассистам. Если точка находится ниже этой линии, это означает, что игрок сделал меньше реальных ассистов, чем ожидалось (недооценка его вклада). Если точка выше линии, то игрок сделал больше ассистов, чем ожидалось (переоценка его вклада).

Точки в верхнем правом углу: Игроки, которые оказались в этом районе, сделали много ассистов, и их вклад (xA) был подтвержден. Это может быть показатель высокой результативности в атакующих действиях.

Точки в нижнем левом углу: Игроки, оказавшиеся здесь, имели ожидаемые ассисты, но на практике они не смогли привести эти моменты к реальным ассистам. Это может быть связано с плохой точностью пасов, неудачными ударами партнеров, или другими факторами.

Точки в верхней части графика, но далеко от линии: Эти игроки создали больше ассистов, чем ожидалось. Это может означать, что они оказались успешными в ситуациях, где было мало шансов (например, точные передачи в моменты, когда шансы на гол были низкими).

##Продвижение мяча — какие игроки Манчестер Сити двигают мяч вперед лучше всех

In [3]:
import sqlite3
import pandas as pd
import plotly.express as px

# Подключение к базе данных
db_path = 'fbref_data.db'
conn = sqlite3.connect(db_path)

# Запрос данных для Манчестер Сити
query = """
SELECT Player, Squad, Total_Passing_Distance, Total_Progressive_Distance,
       Progressive_Passes, Passes_into_Penalty_Area, Passes_into_Final_Third
FROM players_90min_stats
WHERE Squad = 'Manchester City'
"""

df = pd.read_sql(query, conn)
conn.close()

# Преобразование типов
numeric_cols = ['Total_Passing_Distance', 'Total_Progressive_Distance',
                'Progressive_Passes', 'Passes_into_Penalty_Area', 'Passes_into_Final_Third']

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# График 1: Общее продвижение мяча
fig1 = px.scatter(df, x='Total_Passing_Distance', y='Total_Progressive_Distance',
                  hover_data={"Player": True, "Total_Passing_Distance": True, "Total_Progressive_Distance": True},
                  labels={'Total_Passing_Distance': 'Общая дистанция передач (м)',
                          'Total_Progressive_Distance': 'Прогрессивная дистанция передач (м)'})
fig1.update_traces(textposition='top center', hoverinfo='text+x+y')
fig1.show()

График 1: Общее продвижение мяча (за игру)

*   X-ось: сколько метров игрок в сумме продвинул мяч
*   Y-ось: сколько метров мяч двигался в сторону ворот соперника

*   Игроки, находящиеся выше, продвигают мяч ближе к атакующей трети
*   Игроки, расположенные правее, просто выполняют больше передач

Вывод: кто выполняет не просто много передач, но и продвигает мяч эффективно.

In [4]:
# График 2: Количество прогрессивных передач
fig2 = px.bar(df.sort_values(by='Progressive_Passes', ascending=False),
              x='Player', y='Progressive_Passes',
              title='Продвижение мяча: Количество прогрессивных передач')
fig2.show()

График 2: Количество прогрессивных передач (зи игру)

*   Вывод: кто в команде чаще всего продвигает мяч вперед через передачи.

In [5]:
# График 3: Передачи в штрафную
fig3 = px.bar(df.sort_values(by='Passes_into_Penalty_Area', ascending=False),
              x='Player', y='Passes_into_Penalty_Area',
              title='Продвижение мяча: Передачи в штрафную площадь')
fig3.show()

График 3: Передачи в штрафную площадь (зи игру)

*   Показывает кто строит игру и является плей-мейкером
*   Вывод: кто создает моменты и снабжает мячом игроков в голевых зонах.

In [6]:
# График 4: Передачи в последнюю треть поля
fig4 = px.bar(df.sort_values(by='Passes_into_Final_Third', ascending=False),
              x='Player', y='Passes_into_Final_Third',
              title='Продвижение мяча: Передачи в последнюю треть поля')
fig4.show()

График 4: Передачи в последнюю треть поля (за игру)

*   Отражает, кто чаще всего доводит мяч в атакующую треть поля
*   Вывод: кто отвечает за контроль мяча и продвижение к штрафной

Общий вывод:

1.   Если один игрок находится в топе всех 4 графиков — он главный плеймейкер.
2.   Если защитник высоко в графиках 1 и 4, он важен в билдапе.
3.   Если нападающий лидирует в графике 3, он помогает создавать моменты, а не только завершает атаки.

##Кто лучший распасовщик в финальной трети?

In [7]:
import sqlite3
import pandas as pd
import plotly.express as px

# Подключение к базе данных
db_path = 'fbref_data.db'
conn = sqlite3.connect(db_path)

# Запрос данных для Манчестер Сити
query = """
SELECT Player, Squad, Passes_into_Final_Third, Passes_into_Penalty_Area,
       Key_Passes, Assists, Expected_Assists
FROM players_90min_stats
WHERE Squad = 'Manchester City'
"""

df = pd.read_sql(query, conn)
conn.close()

# Преобразование типов
numeric_cols = ['Passes_into_Final_Third', 'Passes_into_Penalty_Area',
                'Key_Passes', 'Assists', 'Expected_Assists']

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# График 1: Сравнение передач в финальную треть и в штрафную площадь
fig1 = px.scatter(df, x='Passes_into_Final_Third', y='Passes_into_Penalty_Area',
                  hover_data={"Player": True, "Passes_into_Final_Third": True, "Passes_into_Penalty_Area": True},
                  labels={'Passes_into_Final_Third': 'Передачи в финальную треть',
                          'Passes_into_Penalty_Area': 'Передачи в штрафную площадь'},
                  title='Передачи в финальную треть vs. Передачи в штрафную площадь')
fig1.show()

График 1: Показывает, кто активно продвигает мяч в финальную треть, но меньше рискует передачами в штрафную.

In [8]:
# График 2: Сравнение ключевых передач и ассистов
fig2 = px.scatter(df, x='Key_Passes', y='Assists',
                  hover_data={"Player": True, "Key_Passes": True, "Assists": True},
                  labels={'Key_Passes': 'Ключевые передачи', 'Assists': 'Ассисты'},
                  title='Ключевые передачи vs. Ассисты')
fig2.add_shape(type="line", x0=0, y0=0, x1=df["Key_Passes"].max(), y1=df["Key_Passes"].max(),
               line=dict(color="red", dash="dash"))
fig2.show()

График 2: Показывает, кто создает много моментов (Key Passes), но получает мало ассистов (Assists). Если игрок находится ниже красной линии, его партнеры плохо реализуют моменты.

In [9]:
# График 3: Сравнение передач в штрафную площадь и ожидаемых ассистов
fig3 = px.scatter(df, x='Passes_into_Penalty_Area', y='Expected_Assists',
                  hover_data={"Player": True, "Passes_into_Penalty_Area": True, "Expected_Assists": True},
                  labels={'Passes_into_Penalty_Area': 'Передачи в штрафную площадь',
                          'Expected_Assists': 'Ожидаемые ассисты (xA)'},
                  title='Передачи в штрафную площадь vs. Ожидаемые ассисты (xA)')
fig3.show()


График 3: Показывает, кто создает больше всего потенциально голевых моментов (xA) через передачи в штрафную.